# Slicing of Kerr spacetime by spacelike submanifolds

This notebook demonstrates a few capabilities of SageMath in computations regarding embedded submanifolds on the specific example of the 3+1 slicing of Kerr spacetime. The corresponding tools have been developed within the  [SageManifolds](https://sagemanifolds.obspm.fr) project (version 1.3, as included in SageMath 8.3).

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v1.3/SM_submanifold_Kerr_slicing.ipynb) to download the notebook file (ipynb format). To run it, you must start SageMath within the Jupyter notebook, via the command `sage -n jupyter`

*NB:* a version of SageMath at least equal to 8.3 is required to run this notebook:

In [1]:
version()

'SageMath version 8.3, Release Date: 2018-08-03'

The first cell defines some notebook options. Parallelism is usefull to speedup calculation of the connection.

In [2]:
%display latex
Parallelism().set(nproc=8)

## Manifolds declaration

Next cells are used to define the manifolds. In our case, $M$ will be the Kerr spacetime, of which $N$ is a single spacelike submanifold of dimension 3 (hypersurface).

In [3]:
M = Manifold(4, 'M', structure="Lorentzian")
print(M)

4-dimensional Lorentzian manifold M


In [4]:
N = Manifold(3, 'N', ambient=M, structure="Riemannian")
print(N)

3-dimensional pseudo-Riemannian submanifold N embedded in 4-dimensional differentiable manifold M


Note the `ambient=M`, used to declare $N$ as a submanifold of $M$.

We will use a single chart, corresponding to the so-called **Kerr coordinates** (*NB*: they differ from the standard Boyer-Lindquist coordinates):

In [5]:
C.<t,r,th,ph> = M.chart(r't:(-oo,+oo) r:(0,+oo) th:(0,pi):\theta ph:(-pi,pi):\phi')

### Spacetime metric

The metric depends on two quantities: the mass of the black-hole and its angular momentum. These are declared as symbolic variables. $\rho$ is a shortcut notation.

In [6]:
m = var('m') # mass
a = var('a') # ratio J/m

In [7]:
rho = sqrt(r^2+a^2*cos(th)^2)

We set the metric components in the Kerr coordinates introduced above:

In [8]:
g = M.metric()
g[0,0], g[1,1], g[2,2] = -(1-2*m*r/rho^2), 1+2*m*r/rho^2, rho^2
g[3,3] = (r^2+a^2+2*a^2*m*r*sin(th)**2/rho^2)*sin(th)^2
g[0,1] = 2*m*r/rho^2
g[0,3] = -2*a*m*r/rho^2*sin(th)^2
g[1,3] = -a*sin(th)^2*(1+2*m*r/rho^2)

In [9]:
g[:]

[                                  2*m*r/(a^2*cos(th)^2 + r^2) - 1                                       2*m*r/(a^2*cos(th)^2 + r^2)                                                                 0                          -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)]
[                                      2*m*r/(a^2*cos(th)^2 + r^2)                                   2*m*r/(a^2*cos(th)^2 + r^2) + 1                                                                 0                    -a*(2*m*r/(a^2*cos(th)^2 + r^2) + 1)*sin(th)^2]
[                                                                0                                                                 0                                               a^2*cos(th)^2 + r^2                                                                 0]
[                         -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)                    -a*(2*m*r/(a^2*cos(th)^2 + r^2) + 1)*sin(th)^2                                                                 0 (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2]

Let's compute the Levi-Civita connection:

In [10]:
nab = g.connection()

List of Christoffel symbols:

In [11]:
nab.display(only_nonredundant=True)

Gam^t_t,t = -2*(a^2*m^2*r*cos(th)^2 - m^2*r^3)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^t_t,r = -(a^4*m*cos(th)^4 + 2*a^2*m^2*r*cos(th)^2 - 2*m^2*r^3 - m*r^4)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^t_t,th = -2*a^2*m*r*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) 
Gam^t_t,ph = 2*(a^3*m^2*r*cos(th)^2 - a*m^2*r^3)*sin(th)^2/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^t_r,r = -2*(a^4*m*cos(th)^4 + a^2*m^2*r*cos(th)^2 - m^2*r^3 - m*r^4)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^t_r,th = -2*a^2*m*r*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) 
Gam^t_r,ph = (a^5*m*cos(th)^4 + 2*a^3*m^2*r*cos(th)^2 - 2*a*m^2*r^3 - a*m*r^4)*sin(th)^2/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^t_th,th = -2*m*r^2/(a^2*cos(th)^2 + r^2) 
Gam^t_th,ph = 2*a^3*m*r*cos(th)*sin(th)^3/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) 
Gam^t_ph,ph = -2*((a^4*m^2*r*cos(th)^2 - a^2*m^2*r^3)*sin(th)^4 + (a^4*m*r^2*cos(th)^4 + 2*a^2*m*r^4*cos(th)^2 + m*r^6)*sin(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^r_t,t = (a^2*m*r^2 - 2*m^2*r^3 + m*r^4 - (a^4*m - 2*a^2*m^2*r + a^2*m*r^2)*cos(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^r_t,r = (2*a^2*m^2*r*cos(th)^2 - 2*m^2*r^3 - (a^4*m*cos(th)^2 - a^2*m*r^2)*sin(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^r_t,ph = -(a^3*m*r^2 - 2*a*m^2*r^3 + a*m*r^4 - (a^5*m - 2*a^3*m^2*r + a^3*m*r^2)*cos(th)^2)*sin(th)^2/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^r_r,r = (2*a^4*m*cos(th)^4 + a^2*m*r^2 - 2*m^2*r^3 - m*r^4 - (a^4*m - 2*a^2*m^2*r + a^2*m*r^2)*cos(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^r_r,th = -a^2*cos(th)*sin(th)/(a^2*cos(th)^2 + r^2) 
Gam^r_r,ph = ((a^5*m*cos(th)^2 - a^3*m*r^2)*sin(th)^4 + (a^5*r*cos(th)^4 + 2*a*m^2*r^3 + a*r^5 - 2*(a^3*m^2*r - a^3*r^3)*cos(th)^2)*sin(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^r_th,th = -(a^2*r - 2*m*r^2 + r^3)/(a^2*cos(th)^2 + r^2) 
Gam^r_ph,ph = ((a^4*m*r^2 - 2*a^2*m^2*r^3 + a^2*m*r^4 - (a^6*m - 2*a^4*m^2*r + a^4*m*r^2)*cos(th)^2)*sin(th)^4 - (a^2*r^5 - 2*m*r^6 + r^7 + (a^6*r - 2*a^4*m*r^2 + a^4*r^3)*cos(th)^4 + 2*(a^4*r^3 - 2*a^2*m*r^4 + a^2*r^5)*cos(th)^2)*sin(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^th_t,t = -2*a^2*m*r*cos(th)*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^th_t,r = -2*a^2*m*r*cos(th)*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^th_t,ph = 2*(a^3*m*r + a*m*r^3)*cos(th)*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^th_r,r = -2*a^2*m*r*cos(th)*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^th_r,th = r/(a^2*cos(th)^2 + r^2) 
Gam^th_r,ph = (a^5*cos(th)^5 + 2*a^3*r^2*cos(th)^3 + (2*a^3*m*r + 2*a*m*r^3 + a*r^4)*cos(th))*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^th_th,th = -a^2*cos(th)*sin(th)/(a^2*cos(th)^2 + r^2) 
Gam^th_ph,ph = -((a^6 - 2*a^4*m*r + a^4*r^2)*cos(th)^5 + 2*(a^4*r^2 - 2*a^2*m*r^3 + a^2*r^4)*cos(th)^3 + (2*a^4*m*r + 4*a^2*m*r^3 + a^2*r^4 + r^6)*cos(th))*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^ph_t,t = -(a^3*m*cos(th)^2 - a*m*r^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^ph_t,r = -(a^3*m*cos(th)^2 - a*m*r^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^ph_t,th = -2*a*m*r*cos(th)/((a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)*sin(th)) 
Gam^ph_t,ph = (a^4*m*cos(th)^2 - a^2*m*r^2)*sin(th)^2/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^ph_r,r = -(a^3*m*cos(th)^2 - a*m*r^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6) 
Gam^ph_r,th = -(a^3*cos(th)^3 + (2*a*m*r + a*r^

Ricci tensor:

In [12]:
nab.ricci()[:]

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]

This proves that the metric given in cell [8] is indeed solution of the vacuum Einstein equation, as expected of a Kerr metric.

It's now time to foliate it along the $t$ axis.

## Slicing:

To compute 3+1 quantities, we must first declare an embedding from $N$ to $M$. because we want our foliation to encompass the whole manifold, the embedding must depend on a parameter $\tau$.

In [13]:
tau = var(r'tau') # foliation parameter

Let's also declare a chart on $N$.

In [14]:
CN.<r0,th0,ph0> = N.chart(r'r0:(0,+oo) th0:(0,pi):\theta_0 ph0:(-pi,pi):\phi_0')

The embedding used is a very simple one, it simply sets $t = \tau$.

In [15]:
phi = N.diff_map(M, {(CN,C):[tau,r0,th0,ph0]})
phi.display()

N --> M
   (r0, th0, ph0) |--> (t, r, th, ph) = (tau, r0, th0, ph0)

The two partial inverses are also quite simple:

In [16]:
phi_inv = M.diff_map(N, {(C,CN):[r,th,ph]})
phi_inv.display()

M --> N
   (t, r, th, ph) |--> (r0, th0, ph0) = (r, th, ph)

In [17]:
phi_inv_tau = M.scalar_field({C:t})
phi_inv_tau.display()

M --> R
(t, r, th, ph) |--> t

The following line registers the embedding, as well as the inverses. This cannot be done during the creation of the manifolds because the embedding `phi` needs exisiting domain and codomain to be created.

In [18]:
N.set_embedding(phi, inverse=phi_inv, var=tau, t_inverse={tau: phi_inv_tau})

Computing the adapted chart is needed for computing the shift and extrinsic curvature. This create another chart, which in our case is identical to `CN` because of the form of the embedding.

In [19]:
T = N.adapted_chart()
T

[Chart (M, (r0_M, th0_M, ph0_M, tau_M))]

The induced metric on $N$ (or first fundamental form) is then given by:

In [20]:
N.induced_metric()[:]

[                                                    (a^2*cos(th0)^2 + 2*m*r0 + r0^2)/(a^2*cos(th0)^2 + r0^2)                                                                                                            0                                     -(a^3*cos(th0)^2 + 2*a*m*r0 + a*r0^2)*sin(th0)^2/(a^2*cos(th0)^2 + r0^2)]
[                                                                                                           0                                                                                        a^2*cos(th0)^2 + r0^2                                                                                                            0]
[                                    -(a^3*cos(th0)^2 + 2*a*m*r0 + a*r0^2)*sin(th0)^2/(a^2*cos(th0)^2 + r0^2)                                                                                                            0 (2*a^2*m*r0*sin(th0)^4 + (a^2*r0^2 + r0^4 + (a^4 + a^2*r0^2)*cos(th0)^2)*sin(th0)^2)/(a^2*cos(th0)^2 + r0^2)]

And its inverse:

In [21]:
N.induced_metric().inverse()[:]

[(2*a^2*m*r0*sin(th0)^2 + a^2*r0^2 + r0^4 + (a^4 + a^2*r0^2)*cos(th0)^2)/(a^4*cos(th0)^4 + 2*m*r0^3 + r0^4 + 2*(a^2*m*r0 + a^2*r0^2)*cos(th0)^2)                                                                                                                                               0                                                                                                                       a/(a^2*cos(th0)^2 + r0^2)]
[                                                                                                                                              0                                                                                                                       1/(a^2*cos(th0)^2 + r0^2)                                                                                                                                               0]
[                                                                                                                      a/(a^2*cos(th0)^2 + r0^2)                                                                                                                                               0                                                                                                   -1/(a^2*sin(th0)^4 - (a^2 + r0^2)*sin(th0)^2)]

The normal vector is given by

In [22]:
N.normal().display()

n = sqrt(a^2*cos(th)^2 + 2*m*r + r^2)/sqrt(a^2*cos(th)^2 + r^2) d/dt - 2*m*r/(sqrt(a^2*cos(th)^2 + 2*m*r + r^2)*sqrt(a^2*cos(th)^2 + r^2)) d/dr

Let's check that it is correcly normalized:

In [23]:
g(N.normal(), N.normal()).display()

g(n,n): M --> R
   (t, r, th, ph) |--> -1
   (r0_M, th0_M, ph0_M, tau_M) |--> -1

The lapse and shift are also easily computed: 

In [24]:
N.lapse().display()

N: M --> R
   (t, r, th, ph) |--> sqrt(a^2*cos(th)^2 + r^2)/sqrt(a^2*cos(th)^2 + 2*m*r + r^2)
   (r0_M, th0_M, ph0_M, tau_M) |--> sqrt(a^2*cos(th0_M)^2 + r0_M^2)/sqrt(a^2*cos(th0_M)^2 + 2*m*r0_M + r0_M^2)

In [25]:
N.shift().display()

beta = 2*m*r/(a^2*cos(th)^2 + 2*m*r + r^2) d/dr

The extrinsic curvature (or second fundamental form) is long to compute because of failed simplifications (see the last component).

In [26]:
N.second_fundamental_form()[:]

[                                                                                                                                                                                                                          2*(a^4*m*cos(th0)^4 + a^2*m^2*r0*cos(th0)^2 - m^2*r0^3 - m*r0^4)/((a^4*cos(th0)^4 + 2*a^2*r0^2*cos(th0)^2 + r0^4)*sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2)*sqrt(a^2*cos(th0)^2 + r0^2))                                                                                                                                                                                                                                                               2*sqrt(a^2*cos(th0)^2 + r0^2)*a^2*m*r0*cos(th0)*sin(th0)/((a^4*cos(th0)^4 + 2*a^2*r0^2*cos(th0)^2 + r0^4)*sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2))                                                                                                                                                                      -(a^5*m*sin(th0)^6 - 2*(a^5*m + a^3*m^2*r0)*sin(th0)^4 + (a^5*m + 2*a^3*m^2*r0 - 2*a*m^2*r0^3 - a*m*r0^4)*sin(th0)^2)/((a^4*cos(th0)^4 + 2*a^2*r0^2*cos(th0)^2 + r0^4)*sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2)*sqrt(a^2*cos(th0)^2 + r0^2))]
[                                                                                                                                                                                                                                                              2*sqrt(a^2*cos(th0)^2 + r0^2)*a^2*m*r0*cos(th0)*sin(th0)/((a^4*cos(th0)^4 + 2*a^2*r0^2*cos(th0)^2 + r0^4)*sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2))                                                                                                                                                                                                                                                                                                                                   2*m*r0^2/(sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2)*sqrt(a^2*cos(th0)^2 + r0^2))                                                                                                                                                                                                                                                                                                          -2*a^3*m*r0*cos(th0)*sin(th0)^3/(sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2)*(a^2*cos(th0)^2 + r0^2)^(3/2))]
[                                                                                                                                                                     -(a^5*m*sin(th0)^6 - 2*(a^5*m + a^3*m^2*r0)*sin(th0)^4 + (a^5*m + 2*a^3*m^2*r0 - 2*a*m^2*r0^3 - a*m*r0^4)*sin(th0)^2)/((a^4*cos(th0)^4 + 2*a^2*r0^2*cos(th0)^2 + r0^4)*sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2)*sqrt(a^2*cos(th0)^2 + r0^2))                                                                                                                                                                                                                                                                                                          -2*a^3*m*r0*cos(th0)*sin(th0)^3/(sqrt(a^2*cos(th0)^2 + 2*m*r0 + r0^2)*(a^2*cos(th0)^2 + r0^2)^(3/2)) -2*((a^4*m^2*r0 - a^4*m*r0^2)*sin(th0)^6 - (a^4*m^2*r0 - 2*a^4*m*r0^2 - a^2*m^2*r0^3 - 2*a^2*m*r0^4)*sin(th0)^4 - (a^4*m*r0^2 + 2*a^2*m*r0^4 + m*r0^6)*sin(th0)^2)*sqrt(-a^2*sin(th0)^2 + a^2 + 2*m*r0 + r0^2)*sqrt(-a^2*sin(th0)^2 + a^2 + r0^2)/(a^8*cos(th0)^8 + 2*m*r0^7 + r0^8 + 2*(a^6*m*r0 + 2*a^6*r0^2)*cos(th0)^6 + 6*(a^4*m*r0^3 + a^4*r0^4)*cos(th0)^4 + 2*(3*a^2*m*r0^5 + 2*a^2*r0^6)*cos(th0)^2)]

## Example of 3+1 projection

It is also possible to project any tensor on $M$ orhogonaly on the submnifold or on the normal vector. This can be useful for example to decompose an energy-impulsion tensor.

Let's construct a fourth-order tensor from a simple vector field.

In [27]:
v = M.vector_field()
v.add_comp()[:] = [1,1,1,1]

The fourth-order tensor $v\otimes v\otimes v\otimes v$ is then projected twice on the submanifold (on the indices 1 and 3) and twice along the normal vector (indicies 0 and 2). The result is the following second-order tensor on $M$:

In [28]:
N.mixed_projection(v*v*v*v, [0,2]).display()

(a^6*cos(th)^6 + 16*m^2*r^4 + 8*m*r^5 + r^6 + (8*a^4*m*r + 3*a^4*r^2)*cos(th)^4 + (16*a^2*m^2*r^2 + 16*a^2*m*r^3 + 3*a^2*r^4)*cos(th)^2)/(a^6*cos(th)^6 + 8*m^3*r^3 + 12*m^2*r^4 + 6*m*r^5 + r^6 + 3*(2*a^4*m*r + a^4*r^2)*cos(th)^4 + 3*(4*a^2*m^2*r^2 + 4*a^2*m*r^3 + a^2*r^4)*cos(th)^2) d/dr*d/dr + (a^4*cos(th)^4 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2)/(a^4*cos(th)^4 + 4*m^2*r^2 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2) d/dr*d/dth + (a^4*cos(th)^4 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2)/(a^4*cos(th)^4 + 4*m^2*r^2 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2) d/dr*d/dph + (a^4*cos(th)^4 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2)/(a^4*cos(th)^4 + 4*m^2*r^2 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2) d/dth*d/dr + (a^2*cos(th)^2 + r^2)/(a^2*cos(th)^2 + 2*m*r + r^2) d/dth*d/dth + (a^2*cos(th)^2 + r^2)/(a^2*cos(th)^2 + 2*m*r + r^2) d/dth*d/dph + (a^4*cos(th)^4 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2)/(a^4*cos(th)^4 + 4*m^2*r^2 + 4*m*r^3 + r^4 + 2*(2*a^2*m*r + a^2*r^2)*cos(th)^2) d/dph*d/dr + (a^2*cos(th)^2 + r^2)/(a^2*cos(th)^2 + 2*m*r + r^2) d/dph*d/dth + (a^2*cos(th)^2 + r^2)/(a^2*cos(th)^2 + 2*m*r + r^2) d/dph*d/dph